In [3]:
import os
import numpy as np
import tensorflow as tf
import json
import time
from typing import Dict

l = tf.keras.layers

In [4]:
def _convert_input_img(img):
    # Decode bytes of image and transform to expected type and shape
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.expand_dims(img, axis=-1)
    return img

def _tuple_to_feat_label(x, y):
    x = _convert_input_img(x)
    y = tf.cast(y, dtype=tf.int32)
    return { 'image': x }, y
  
def make_dataset(xs, ys, batch_size=1024, shuffle=False, num_epochs=None):
  
    dataset = tf.data.Dataset.from_tensor_slices((xs, ys))
    if shuffle:
        dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(
            buffer_size=4096, count=num_epochs))
    else:
        dataset = dataset.repeat(num_epochs)

    dataset = dataset.apply(tf.contrib.data.map_and_batch(
        _tuple_to_feat_label, batch_size=batch_size))

    return dataset

In [5]:
def mnist_cnn_model_fn(features: Dict[str, tf.Tensor], labels: tf.Tensor, mode: tf.estimator.ModeKeys, params: dict):

    is_training = mode == tf.estimator.ModeKeys.TRAIN

    dropout_rate = 0
    if is_training:
        dropout_rate = params.get('dropout_rate', 0.4)

    learning_rate = params.get('learning_rate', 1e-3)
    decay_rate = params.get('decay_rate', 1e-6)
    decay_steps = params.get('decay_steps', 500)

    conv1 = l.Conv2D(filters=32, kernel_size=5, padding='same', activation=tf.nn.relu)(features['image'])
    pool1 = l.MaxPooling2D(pool_size=2, strides=2, padding='same')(conv1)

    conv2 = l.Conv2D(filters=64, kernel_size=5, padding='same', activation=tf.nn.relu)(pool1)
    pool2 = l.MaxPooling2D(pool_size=2, strides=2, padding='same')(conv2)

    pool2_flat = l.Flatten()(pool2)
    dense = l.Dense(1024, activation=tf.nn.relu)(pool2_flat)
    dropout = l.Dropout(rate=dropout_rate)(dense)

    logits = l.Dense(10)(dropout)
  
    head = tf.contrib.estimator.multi_class_head(n_classes=10)
  
    decayed_learning_rate = tf.train.exponential_decay(
        learning_rate, tf.train.get_global_step(), 
        decay_steps, decay_rate, staircase=True)

    optimizer = tf.train.AdamOptimizer(decayed_learning_rate)
  
    return head.create_estimator_spec(
        features, mode, logits, labels, optimizer=optimizer
    )

In [6]:
def serving_input_receiver_fn():
    """Input receiver expects a batched 28x28 uint8 array of pixels"""

    # Define inputs from serving receiver
    inputs = { 
        'image': tf.placeholder(tf.uint8, shape=[None, 28, 28]) 
    }    
    
    # Define features passed to model_fn
    features = {
        'image': tf.map_fn(_convert_input_img, inputs['image'], dtype=tf.float32)
    }
    
    return tf.estimator.export.ServingInputReceiver(features, inputs)

In [10]:
def run():

    tf.logging.set_verbosity(tf.logging.INFO)
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    train_ds = lambda: make_dataset(x_train, y_train, shuffle=True)
    test_ds = lambda: make_dataset(x_test, y_test)

    estimator = tf.estimator.Estimator(
        mnist_cnn_model_fn, 
        model_dir=os.environ.get('MODEL_DIR', '/tmp/mnist'), 
        params={
            'learning_rate': 1e-3,
            'dropout_rate': 0.4,
        })

    max_steps = int(os.environ.get('MAX_STEPS', 3))
    train_spec = tf.estimator.TrainSpec(train_ds, max_steps=max_steps)
    
    final_exporter = tf.estimator.FinalExporter('mnist', serving_input_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(test_ds, exporters=[final_exporter])
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125406898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen aft

In [11]:
!saved_model_cli show --dir /tmp/mnist/export/mnist/1538436806 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image'] tensor_info:
        dtype: DT_UINT8
        shape: (-1, 28, 28)
        name: Placeholder:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['class_ids'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: head/predictions/ExpandDims:0
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 1)
        name: head/predictions/str_classes:0
    outputs['logits'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: dense_1/BiasAdd:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: head/predictions/probabilities:0
  Method name is: tensorflow/serving/predict
